In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
from sklearn.cluster import KMeans, DBSCAN
from skimage import io
import os
import tqdm
from sklearn.preprocessing import MinMaxScaler
from collections import Counter

In [6]:
list_dir = os.listdir('data')
list_dir

['.ipynb_checkpoints',
 'KRA_ADMIN_100m.tif',
 'KRA_LANDCOVER_100m.tif',
 'KRA_PREC_100m.tif',
 'KRA_RELIEF_ASPECT_100m.tif',
 'KRA_RELIEF_HEIGHT_100m.tif',
 'KRA_RELIEF_SLOPE_100m.tif',
 'KRA_SOILTEXTURE_100m.tif',
 'KRA_SUNNY_DAYS_APR_OCT_100m.tif',
 'KRA_TAVG_100m.tif',
 'KRA_TMAX_100m.tif',
 'KRA_TMIN_100m.tif',
 'KRA_VINEYARDS_WITH_NUMBERS_100m.tif',
 'KRA_WATER_SEASONALYTY_100m.tif',
 'README.txt',
 'tmp.csv',
 'tmp_off_mask.csv',
 'VINEYARDS.geojson',
 'VINEYARDS.TXT']

# area clustering

In [8]:
img = io.imread('data/KRA_VINEYARDS_WITH_NUMBERS_100m.tif')

In [22]:
xy = np.vstack(np.where(img > 0)).T
x, y = np.where(img > 0)
xy_data = pd.DataFrame(xy, columns = ['x', 'y'])
xy_data['label'] = img[x, y]

In [23]:
xy_data

,x,y,label
0,197,2445,60
1,197,2446,60
2,198,2442,60
3,198,2443,60
4,198,2444,60
...,...,...,...
35581,2938,1363,79
35582,2938,1364,79
35583,2939,1362,79
35584,2939,1363,79


In [25]:
label = np.unique(xy_data['label'])

In [61]:
xy_data = pd.read_csv('cluster_point.csv')

In [63]:
#xy_data = xy_data[['x', 'y', 'pred_batch']]
xy_data = xy_data.rename({'pred_batch' : 'label'}, axis = 1)

In [64]:
label = np.unique(xy_data['label'])

In [65]:
dict_to_rename_label = {v : ind for ind, v in enumerate(label)}

In [81]:
import pickle 
with open('hui.pickle', 'wb') as f:
    pickle.dump(dict_to_rename_label, f)

In [66]:
xy_data['x'] = xy_data['x'].astype(np.int64)
xy_data['y'] = xy_data['y'].astype(np.int64)

In [67]:
xy_data['label'] = xy_data['label'].map(dict_to_rename_label)

In [70]:
xy_data = xy_data[['x', 'y', 'label']]

In [73]:
label = np.unique(xy_data['label'])

# clustering information to need terretory

In [74]:
num_data = ['KRA_RELIEF_ASPECT_100m.tif',
            'KRA_RELIEF_HEIGHT_100m.tif',
            'KRA_RELIEF_SLOPE_100m.tif',
            'KRA_SUNNY_DAYS_APR_OCT_100m.tif',
           'KRA_WATER_SEASONALYTY_100m.tif']

num_many_chanel_data = ['KRA_TAVG_100m.tif',
            'KRA_TMIN_100m.tif',
            'KRA_TMAX_100m.tif',
            'KRA_PREC_100m.tif']
path_to_save = 'to_save/'
cat_data = ['KRA_SOILTEXTURE_100m.tif']
path = 'data/'
uniq_label = np.unique(label)

#num feat
for name in num_data:
    img_tmp = io.imread(path + name).astype(np.float32)
    name_for_write = name.split('.')[0]
    res_list = []
    for cluster in tqdm.tqdm(uniq_label):
        res_dict = {}
        res_dict['index_cluster'] = cluster
        xy_tmp = xy_data[xy_data['label'] == cluster][['x', 'y']].values
        x, y = xy_tmp[:,0], xy_tmp[:,1]
        pixel_wise_value = img_tmp[x,y]
        min_val, max_val, mean_val = pixel_wise_value.min(), pixel_wise_value.max(), pixel_wise_value.mean()
        res_dict['min_value_' + name_for_write] = min_val
        res_dict['max_value_' + name_for_write] = max_val
        res_dict['mean_value_' + name_for_write] = mean_val
        res_list.append(res_dict)
    df_res = pd.DataFrame(res_list, dtype = np.float64)
    df_res.to_csv(path_to_save + name_for_write + '.csv', index = False)



100%|██████████████████████████████████████████████████████████████████████████████| 480/480 [00:00<00:00, 2368.39it/s]


In [75]:
#many chanel data
for name in num_many_chanel_data:
    img_tmp_ = io.imread(path + name).astype(np.float32)
    name_for_write = name.split('.')[0]
    res_list = []
    
    for cluster in tqdm.tqdm(uniq_label):
        res_dict = {}
        res_dict['index_cluster'] = cluster
        for chanel in range(img_tmp_.shape[2]):
            img_tmp = img_tmp_[:,:,chanel]
            xy_tmp = xy_data[xy_data['label'] == cluster][['x', 'y']].values
            x, y = xy_tmp[:,0], xy_tmp[:,1]
            pixel_wise_value = img_tmp[x,y]
            min_val, max_val, mean_val = pixel_wise_value.min(), pixel_wise_value.max(), pixel_wise_value.mean()
            res_dict['min_value_' + name_for_write + '_chanel_' + str(chanel)] = min_val
            res_dict['max_value_' + name_for_write + '_chanel_' + str(chanel)] = max_val
            res_dict['mean_value_' + name_for_write + '_chanel_' + str(chanel)] = mean_val
        res_list.append(res_dict)
    df_res = pd.DataFrame(res_list, dtype = np.float64)
    df_res.to_csv(path_to_save + name_for_write + '.csv', index = False)

100%|███████████████████████████████████████████████████████████████████████████████| 480/480 [00:02<00:00, 184.38it/s]


In [76]:
for name in cat_data:
    img_tmp = io.imread(path + name).astype(np.float32)
    name_for_write = name.split('.')[0]
    res_list = []
    uniq_val = np.unique(img_tmp)
    
    for cluster in tqdm.tqdm(uniq_label):
        res_dict = {}
            
        res_dict['index_cluster'] = cluster
        xy_tmp = xy_data[xy_data['label'] == cluster][['x', 'y']].values
        x, y = xy_tmp[:,0], xy_tmp[:,1]
        pixel_wise_value = img_tmp[x,y]
        c = Counter(pixel_wise_value)
        for v in uniq_val:
            res_dict['count_of_' + str(v)] = c[v]
        res_list.append(res_dict)
            
    df_res = pd.DataFrame(res_list)
    df_res.to_csv(path_to_save + name_for_write + '.csv')

100%|██████████████████████████████████████████████████████████████████████████████| 480/480 [00:00<00:00, 2331.70it/s]


In [77]:
list_dir = os.listdir('to_save')

In [78]:
all_df = pd.DataFrame(label, columns = ['index_cluster'])
for f in list_dir:
    try:
        df = pd.read_csv('to_save/' + f)
    except:
        continue
        
    all_df = all_df.merge(df, on = 'index_cluster')


In [79]:
all_df.to_csv('freadom.csv', index = False)

In [162]:
df = pd.read_csv('to_save/' + f)

In [51]:
df

,index_cluster,min_value_KRA_PREC_100m_chanel_0,max_value_KRA_PREC_100m_chanel_0,mean_value_KRA_PREC_100m_chanel_0,min_value_KRA_PREC_100m_chanel_1,max_value_KRA_PREC_100m_chanel_1,mean_value_KRA_PREC_100m_chanel_1,min_value_KRA_PREC_100m_chanel_2,max_value_KRA_PREC_100m_chanel_2,mean_value_KRA_PREC_100m_chanel_2,...,mean_value_KRA_PREC_100m_chanel_8,min_value_KRA_PREC_100m_chanel_9,max_value_KRA_PREC_100m_chanel_9,mean_value_KRA_PREC_100m_chanel_9,min_value_KRA_PREC_100m_chanel_10,max_value_KRA_PREC_100m_chanel_10,mean_value_KRA_PREC_100m_chanel_10,min_value_KRA_PREC_100m_chanel_11,max_value_KRA_PREC_100m_chanel_11,mean_value_KRA_PREC_100m_chanel_11
0,1000200.0,48.0,48.0,48.0,39.0,39.0,39.0,39.0,39.0,39.000000,...,48.000000,41.0,41.0,41.000000,49.0,49.0,49.0,62.0,62.0,62.000000
1,1008000.0,49.0,49.0,49.0,40.0,40.0,40.0,40.0,40.0,40.000000,...,48.000000,41.0,41.0,41.000000,50.0,50.0,50.0,63.0,63.0,63.000000
2,1008600.0,49.0,49.0,49.0,40.0,40.0,40.0,40.0,41.0,40.428570,...,46.142857,40.0,40.0,40.000000,50.0,50.0,50.0,62.0,62.0,62.000000
3,1016900.0,49.0,49.0,49.0,40.0,40.0,40.0,40.0,40.0,40.000000,...,48.000000,41.0,41.0,41.000000,50.0,50.0,50.0,62.0,63.0,62.166668
4,1019200.0,49.0,49.0,49.0,40.0,40.0,40.0,40.0,40.0,40.000000,...,47.000000,41.0,41.0,41.000000,50.0,50.0,50.0,63.0,63.0,63.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,963600.0,48.0,48.0,48.0,39.0,39.0,39.0,38.0,38.0,38.000000,...,49.000000,40.0,40.0,40.000000,49.0,49.0,49.0,62.0,62.0,62.000000
476,971400.0,48.0,48.0,48.0,39.0,40.0,39.5,38.0,38.0,38.000000,...,47.000000,40.0,40.0,40.000000,50.0,50.0,50.0,62.0,62.0,62.000000
477,974700.0,48.0,48.0,48.0,39.0,39.0,39.0,38.0,38.0,38.000000,...,49.000000,40.0,41.0,40.833332,49.0,49.0,49.0,62.0,62.0,62.000000
478,990600.0,48.0,48.0,48.0,39.0,39.0,39.0,38.0,39.0,38.384617,...,48.000000,41.0,41.0,41.000000,49.0,49.0,49.0,62.0,62.0,62.000000


In [52]:
all_df

,index_cluster
0,10002_00
1,10080_00
2,10086_00
3,10169_00
4,10192_00
...,...
475,9636_00
476,9714_00
477,9747_00
478,9906_00
